In [1]:
import os
import sys
import numpy as np
import torch
import plotly.graph_objects as go
import tqdm
import pickle
import kaleido

ROOT = os.path.join("./")
sys.path.append(ROOT + "lib")

from plotly_style import update_layout

buckets = pickle.load(open(ROOT + "cached_data/raw_classes.pkl", "rb"))

In [2]:
def get_trace(lightsource, key):

  x_data = lightsource["norm"]["day"]
  y_data = lightsource["norm"][key]

  sort_idxs = np.argsort(x_data)

  x_data = x_data[sort_idxs]
  y_data = y_data[sort_idxs]


  return go.Scatter(
    x=x_data,
    y=y_data,
    mode='markers',
    marker=dict(size=5, opacity=.75),
    name=key,
  )

def plot_from_lightsource(lightsource, ys=["w1"]):

  fig = go.Figure()

  for y in ys:
    fig.add_trace(get_trace(lightsource, y))

  update_layout(fig, legend_out=True)

  fig.layout.width = 800
  fig.layout.height = 0.65 * fig.layout.width

  return fig

def plot_from_tensor(data):
  fig = go.Figure()

  w1 = data[:, 0].numpy()
  std = data[:, 1].numpy()


  day = data[:, -1].numpy()


  fig.add_trace(go.Scatter(x=day, y=w1, marker=dict(size=5, opacity=0.7), name="w1mpro z-scored", mode='markers'))
  fig.add_trace(go.Scatter(x=day, y=std, marker=dict(size=5, opacity=0.7), name="w2mpro z-scored", mode='markers'))

  update_layout(fig, legend_out=True)

  fig.layout.width = 800
  fig.layout.height = 0.65 * fig.layout.width


  return fig

In [ ]:
# ----------------------- Begin Work on Dataset ----------------------- # 
ex = buckets["null"]["null21"]
flux = ex["raw"]["w1flux"]

ADJ = 0
DIV = 0.001
norm = np.arcsinh((flux - ADJ) /DIV)
tr1 = go.Scatter(
    x=ex["norm"]["day"],
    y=norm,
    marker=dict(size=5, opacity=0.7), name="normalized flux", mode='markers'
)

tr2 = go.Scatter(
    x=ex["norm"]["day"],
    y=flux,
    marker=dict(size=5, opacity=0.7), name="raw flux", mode='markers'
)

fig = plot_from_lightsource(ex, ["w1"])
fig.add_trace(tr1)
fig.add_trace(tr2)
fig.show()

print([n for n in flux])


In [20]:
# ----------------------- Begin Work on Dataset ----------------------- # 
ex = buckets["pulsating_var"]["BB Lep"]
flux = ex["raw"]["w1flux"]

ADJ = 0
DIV = 0.001
norm = np.arcsinh((flux - ADJ) /DIV)
tr1 = go.Scatter(
    x=ex["norm"]["day"],
    y=norm,
    marker=dict(size=5, opacity=0.7), name="normalized flux", mode='markers'
)

tr2 = go.Scatter(
    x=ex["norm"]["day"],
    y=flux,
    marker=dict(size=5, opacity=0.7), name="raw flux", mode='markers'
)

fig = plot_from_lightsource(ex, ["w1"])
fig.add_trace(tr1)
fig.add_trace(tr2)
fig.show()

print([n for n in flux])


[0.011987176089060886, 0.011259444101264686, 0.0102027397476806, 0.012975278270831486, 0.010155861346200747, 0.013059202865678457, 0.013095339803086374, 0.01311948279015509, 0.012598442129257568, 0.011932098760107118, 0.011342707091560275, 0.011023427808588506, 0.012691606938755587, 0.013289746280865242, 0.01275019007860143, 0.010962682498447078, 0.012951400654578143, 0.011521706197688003, 0.01090226235307003, 0.01321651227656553, 0.01276194360021852, 0.012131561352832063, 0.011542947983538781, 0.01116648888258832, 0.010440388417670218, 0.012679929333564993, 0.013204340105334274, 0.01310741149445018, 0.012691606938755587, 0.011176782499205751, 0.012494443444810854, 0.01265659523001617, 0.01161760821144587, 0.011053931425450627, 0.013204340105334274, 0.012856317494398476, 0.012277696518139104, 0.011606918797362458, 0.01050791724824395, 0.010952586064097956, 0.013035170808092284, 0.013326521169383205, 0.012939484012009618, 0.012391292373389687, 0.011064121283104685, 0.012726726678472376,